In [ ]:
import sqlite3
import typer
from rich import print
import requests
from bs4 import BeautifulSoup

In [ ]:
# Send a GET request to the URL
url = 'https://www.londonambulance.nhs.uk/working-for-us/volunteering-with-us/emergency-responders/'
response = requests.get(url)

In [ ]:
if response.status_code == 200:
    # Return the HTML content of the page
    print(response.text[:500])
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")


In [ ]:
soup = BeautifulSoup(response.content, 'html.parser')
title = soup.title.string if soup.title else 'No Title'
body = soup.find('body').get_text()
print(f"Title: {title}")

In [ ]:
soup.find('body').get_text()
# Remove all \n characters
body = body.replace('\n', ' ')
body = body.replace('\t', ' ')
body = body.split('  ')
body = ''.join(body)
body

In [ ]:
api_key = ""

In [ ]:
from openai import OpenAI

client = OpenAI(api_key=api_key)

response = client.responses.create(
    model="gpt-3.5-turbo",
    input="Write a one-sentence bedtime story about a unicorn."
)

print(response.output_text)

In [ ]:
client = OpenAI(api_key=api_key)

def get_embedding(text, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

get_embedding("We are lucky to live in an age in which we are still making discoveries.")